# 기초확인용

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
train = pd.read_csv("datasets/train.csv")
test = pd.read_csv("datasets/test.csv")
df = pd.concat([train,test])
df['credit'].fillna(-999)

0         1.0
1         1.0
2         2.0
3         0.0
4         2.0
        ...  
9995   -999.0
9996   -999.0
9997   -999.0
9998   -999.0
9999   -999.0
Name: credit, Length: 36457, dtype: float64

In [44]:
train.drop(['index','FLAG_MOBIL'],axis=1,inplace = True)

In [3]:
train.columns

Index(['index', 'gender', 'car', 'reality', 'child_num', 'income_total',
       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'begin_month', 'credit'],
      dtype='object')

# occyp_type credit 평균치로 환산해보기

In [45]:
train['occyp_type'].fillna('OTHER',inplace = True)
occyp_mean_score = train.groupby(['occyp_type']).mean()['credit']
train['occyp_mean_score'] = train['occyp_type'].apply(lambda x : occyp_mean_score[x] )

# hosue_type credit 평균치로 환산해보기

In [49]:
house_mean_score = train.groupby(['house_type']).mean()['credit']
train['house_mean_score'] = train['house_type'].apply(lambda x : house_mean_score[x] )

# 가족관련 늘리기
1. 전체소득 / 가족수 --> 인당 인컴 수준
2. family_size - childnum --> 어른 수 linear해서 큰의미가 없을 것 같긴하지만 3을 위해서 활용
3. 전체소득 / 2에서 나온 것 --> 어른당 얼마정도 사용가능한지 연산

In [50]:
train['total_income/famliy_size'] = train['income_total']/train['family_size']
train['adult'] = train['family_size'] - train['child_num']
train['total_income/adult'] = train['income_total'] / train['adult']

# 발급일 관련 늘리기
1. when_begin_age = begin_month / Days_birth --> 나이대 대비 얼마쯤에 발급한 것인지
2. when_begin_employed = begin_month / Employed --> 고용된 것 대비 얼마만에 발급한 것인지 
3. begin_income =   begin_month / incometotal--> 버는 것 대비 빌린거는 얼마나 되었는지 --> 좀 이상하긴하네..
- 음.. 전처리부분이 고려가 안되어서 그거 고려해서 생각해야할 듯 함 일단 늘리기만 하기 오늘은 (employed - 3만 얼마인가 있음)
- 그리고 얘들은 구간을 나눠서 대표값으로 처리해버리면 더 안좋아질 수도 있을 것 같긴함 

In [68]:
train['when_begin_age'] = train['begin_month'] / train['DAYS_BIRTH']
train['when_begin_employed'] = train['begin_month'] / train['DAYS_EMPLOYED']
train['begin_income'] = train['begin_month'] / train['income_total']

# 나이대비 늘리기
전처리에서 구간별로 나눠진 나이를 활용하면 아래의 적용이 힘들 것 같고 아래의 식은 기존의 컬럼 활용 구간별로 적용은 따로 한개 더 만드는게 좋을 듯 
1. age_income = income_total / DAYS_BIRTH --> 나이대비 얼마나 버는지
2. age_employed = DAYS_EMPLOYED / DAYS_BIRTH --> 일한지는 살아생전에 얼만마 대는지?
3. DAYS_BIRTH / 365 --> 나이 연률화

In [65]:
train['income_age'] = train['income_total'] / train['DAYS_BIRTH']
train['employed_age'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']
train['yearly_age'] = train['DAYS_BIRTH']//365 

# 고용대비 늘리기 
전처리 부분에서 0,1로 고용유무로 구분하는 컬럼하나와 그냥 두는 컬럼 2개다 활용하는 것이 좋은 듯 함  고용기간 별구간나누는 것 까지는 잘 모르겠음 
1. income_total / employed  --> 고용된 것 대비 얼마나 버는지
2. DAYS_EMPLOYED /365 연률화

In [67]:
train['income_employed'] = train['income_total'] / train['DAYS_EMPLOYED']
train['yearly_employed'] = train['DAYS_EMPLOYED'] // 365

#  & 로 늘리기
1. 차와 집 동시 소유 - 전처리에서 Y =1 N =0 으로 전처리 필요
2. 이메일과 전화 동시 소유 -> 그래도 가지고 있는애들이 소득이 조금 높아보이긴 하기 떄문에 활용

In [61]:
train['car&home'] = train['car'] * train['reality']
train['email&phone'] = train['email'] * train['phone']

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,occyp_mean_score,house_mean_score,total_income/famliy_size,adult,total_income/adult,when_begin_age,when_begin_employed,income_age,employed_age,income_employed
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,...,1.526129,1.535452,101250.0,2.0,101250.0,0.000432,0.001274,-14.569393,0.338801,-43.002761
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,...,1.500443,1.521625,82500.0,2.0,123750.0,0.000439,0.003247,-21.748682,0.135325,-160.714286
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,...,1.530226,1.521625,225000.0,2.0,225000.0,0.001153,0.004962,-23.576256,0.232305,-101.488498
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,...,1.513194,1.521625,101250.0,2.0,101250.0,0.002452,0.017686,-13.421262,0.138653,-96.797323
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,...,1.530226,1.521625,78750.0,2.0,78750.0,0.001729,0.012352,-10.474164,0.139988,-74.821853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,...,1.502646,1.521625,56250.0,2.0,112500.0,0.000166,0.001008,-18.627370,0.164252,-113.407258
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,...,1.526129,1.521625,90000.0,1.0,180000.0,0.003074,0.018990,-11.771630,0.161860,-72.727273
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,...,1.502646,1.510740,146250.0,2.0,146250.0,0.002480,0.012407,-29.012101,0.199861,-145.161290
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,...,1.500443,1.521625,171000.0,1.0,171000.0,0.005816,0.551402,-16.855594,0.010547,-1598.130841


# 총소득 늘리기
1. total_earned_money = income_total x (DAYS_EMPLOYED / 365) --> 이떄까지 번 것, DAYS_EMPLOYED가 양수로 바뀔 필요있을 것 같음
2. total/family = total_earned_money / family_num
3. total/age = total_earned_money / DAYS_BIRTH --> 나이대비 이때까지 얼마 벌었는지 
4. log(income_total) --> 소득의 비선형성 고려 

In [77]:
train['total_earned_money'] = train['income_total'] * (train['DAYS_EMPLOYED'] / 365)
train['total/family'] = train['total_earned_money'] / train['family_size']
train['total/age'] = train['total_earned_money'] /train['DAYS_BIRTH']
train['log_income'] = np.log(train['income_total'])

In [79]:
train.corr()['credit']

child_num                   0.004081
income_total                0.008555
DAYS_BIRTH                 -0.025187
DAYS_EMPLOYED              -0.003798
work_phone                 -0.003134
phone                       0.003452
email                       0.014812
family_size                 0.008227
begin_month                -0.147477
credit                      1.000000
occyp_mean_score            0.036839
house_mean_score            0.018901
total_income/famliy_size    0.005924
adult                       0.010627
total_income/adult          0.007196
when_begin_age              0.128102
when_begin_employed         0.036738
income_age                  0.001942
employed_age                0.005066
income_employed             0.014487
yearly_age                 -0.025041
yearly_employed            -0.003814
begin_income               -0.109365
total_earned_money         -0.002643
total/family               -0.001163
total/age                   0.004453
log_income                  0.005539
N